In [737]:
import numpy as np
import scipy.sparse as sp

### Preliminaries

We will be using the `load_as_csr`, `num_rows`, and `matvec` functions from homework #1. They are provided below for you to use.

In [738]:
def load_as_csr(filename):
    # Read m, n, and nnz from the file
    m, n, nnz = np.genfromtxt(filename, max_rows=1, dtype=None)
    # Read the data as a list of tuples [(i,j,v), ...]
    data = np.genfromtxt(filename, skip_header=1, dtype=None)
    # Allocate arrays
    I = np.zeros(m + 1, dtype=int)
    J = np.zeros(nnz, dtype=int)
    V = np.zeros(nnz)
    
    for k in range(nnz):
        i = data[k][0]
        I[i+1] += 1
        
    for i in range(m):
        I[i+1] += I[i]
    
    for k in range(nnz):
        i, j, v = data[k]
        idx = I[i]
        I[i] += 1
        J[idx] = j
        V[idx] = v
    
    for i in reversed(range(m)):
        I[i+1] = I[i]
        
    I[0] = 0
    
    return I, J, V

def num_rows(A):
    return len(A[0]) - 1

def matvec(A, v):
    I, J, V = A
    m = num_rows(A)
    w = np.zeros(m)
    
    for i in range(m):
        begin = I[i]
        end = I[i+1]
        for idx in range(begin, end):
            j = J[idx]
            w[i] += V[idx]*v[j]
    
    return w

### 1. Sparse triangular solves (5 points)

Write functions `tril_solve` and `triu_solve` that take a matrix $A$ in CSR format (as a tuple of three arrays, the output of `load_as_csr`) and a right-hand side vector $b$, and returns the solution to $L x = b$ and $U x = b$, where $L$ and $U$ are the lower and upper triangular parts of $A$, respectively.

In [739]:
def tril_solve(A, b):
    I, J, V = A
    x = np.zeros(len(b))
    
    # Perform lower-triangular solve (forward substitution)
    for i in range(len(b)):
        sum_val = 0.0
        diag_val = 1
        for idx in range(I[i], I[i+1]):
            j = J[idx]
            if j < i:
                sum_val += V[idx] * x[j]
            elif j == i:
                diag_val = V[idx]
        x[i] = (b[i] - sum_val) / diag_val
    return x

def triu_solve(A, b):
    I, J, V = A
    x = np.zeros(len(b))
    
    # Perform upper-triangular solve (backward substitution)
    for i in reversed(range(len(b))):
        sum_val = 0.0
        diag_val = 1
        for idx in range (I[i], I[i+1]):
            j = J[idx]
            if j > i:
                sum_val += V[idx] * x[j]
            elif j == i:
                diag_val = V[idx]
        x[i] = (b[i] - sum_val) / diag_val
    return x

The code below can be used to check your results.

In [740]:
def to_scipy(A):
    return sp.csr_matrix((A[2], A[1], A[0]))

def triu_solve_scipy(A, b):
    return sp.linalg.spsolve(sp.triu(to_scipy(A), format="csr"), b)

def tril_solve_scipy(A, b):
    return sp.linalg.spsolve(sp.tril(to_scipy(A), format="csr"), b)

for filename in ["16.txt", "25.txt", "50.txt", "64.txt"]:
    A = load_as_csr(filename)
    n = num_rows(A)
    b = np.random.default_rng(0).random(n)
    x_l_1 = tril_solve(A, b)
    x_l_2 = tril_solve_scipy(A, b)

    if np.linalg.norm(x_l_1 - x_l_2) < 1e-10:
        print("tril_solve OK")
    else:
        print("tril_solve incorrect")

    x_u_1 = triu_solve(A, b)
    x_u_2 = triu_solve_scipy(A, b)

    if np.linalg.norm(x_u_1 - x_u_2) < 1e-10:
        print("triu_solve OK")
    else:
        print("triu_solve incorrect")

tril_solve OK
triu_solve OK
tril_solve OK
triu_solve OK
tril_solve OK
triu_solve OK
tril_solve OK
triu_solve OK


### 1b. (2 points)

What is the computational complexity of the above functions (in terms of the size of the matrix and the number of nonzeros)? Justify your answer.

### 2. Gauss-Seidel method (5 points)

Write a function `gauss_seidel` that takes a CSR matrix $A$, right-hand side $b$, and implements the Gauss-Seidel method. The string parameter `uplo` will determine if the lower or upper part of the matrix will be used: a value of `"L"` indicates the lower triangular part, and `"U"` indicates the upper triangular part. Use the functions `matvec`, `tril_solve`, and `triu_solve` from above.

Perform a maximum of 1000 iterations, and terminate when the norm of the residual (computed using `np.linalg.norm`) is below `1e-5`.

When the method converges, return the solution, and print the message `"Converged after {i} iterations"`, where `{i}` denotes the number of iterations.

Use the code below to run Gauss-Seidel and output the number of iterations.

In [741]:
# 1b. What is the computational complexity of the above functions (in terms of the size of the matrix and the number of nonzeros)?
# The trill_solve function solves the equations of the matrix where all elements above the diagonal are zero(0). The triu_solve function 
# solves equations of the matricies where all elements below the main diagonal are zero(0). So the work in opposite ways, yet they both check through the 
# non-zero pieces to find the value of a certain component in that row. The matrix(cies) here are 'nnz', and because each non-zero piece is looked at once,
# and iteration also occurs within the indicies I[i] and I[i+1], also looking at the i-th row, the complexity here is O(nnz) for trill_solve and triu_solve functions.
# For the load_as_csr function, the complexity would be O(nnz + m) as both m (number of rows) and nnz (number of non-zero entries) work in different parts of the function 
# (allocating arrays, filling them up, etc.). The matvec function would have a complexity of O(nnz) as the operation depends on the entries of non-zero which
# move accross each row once, making this the complexity.


def gauss_seidel(A, b, uplo="L"):
    I, J, V = A
    x = np.zeros(len(b))  # Initialize solution vector with zeros
    max_iter = 1000       # Maximum iterations
    acc = 1e-5            # Norm of the residual

    
    # Perform Gauss-Seidel iteration  
    for iter_count in range(max_iter):
        x_old = x.copy()
        
        for i in range(len(b)):  # Iterates over the rows of the matrix
            sum_ax = 0  # This will store the sum of A*x for the i-th row

            # Calculate the sum of the row, without diagonal
            for j in range(I[i], I[i + 1]):
                if J[j] != i:
                    sum_ax += V[j] * x[J[j]]

            # Find the diagonal coefficient
            diag_idx = np.where(J[I[i]:I[i + 1]] == i)[0]
            if len(diag_idx) > 0:
                diag_idx += I[i]  # Correcting the index offset(mistake)
                diag_val = V[diag_idx[0]]
            else:
                continue  # Skip if no diagonal element

            # Update x[i] based on whether we are using L or U
            if uplo == "L" and J[diag_idx[0]] <= i:
                x[i] = (b[i] - sum_ax) / diag_val
            elif uplo == "U" and J[diag_idx[0]] >= i:
                x[i] = (b[i] - sum_ax) / diag_val

        # Check for convergence
        if np.linalg.norm(x - x_old) < acc:
            print(f"Converged after {iter_count + 1} iterations")
            return x

    print("Did not converge within the maximum number of iterations")
    return x

In [742]:
for filename in ["16.txt", "25.txt", "50.txt", "64.txt"]:
    A = load_as_csr(filename)
    b = np.random.default_rng(0).random(num_rows(A))
    gauss_seidel(A, b)

Converged after 136 iterations
Converged after 71 iterations
Converged after 115 iterations
Converged after 96 iterations


### 3. Modify the matrices and check convergence (5 points)

Write a function `add_to_diagonal` that takes a CSR matrix $A$ (the output of `load_as_csr`), and returns a **new** matrix, that has the value `alpha` added to the diagonal entries, i.e. $A + \alpha I$.

Re-run the tests above for matrices $A + \alpha I$, with $\alpha \in \{0, 1, 2, 4, 8\}$. Output iteration counts for each test matrix, and for each value of $\alpha$.

What happens for larger $\alpha$? Can you explain this in terms of the eigenvalues of the matrix?

In [743]:
def add_to_diagonal(A, alpha):
    I, J, V1 = A
    V2 = np.copy(V1)
    n = num_rows(A)

    # Add alpha to the diagonal entries
    for i in range(len(I) - 1):
        for idx in range(I[i], I[i+1]):
            if J[idx] == i:  # Only if entry it is on now is on the diagonal
                V2[idx] += alpha
    
    return I, J, V2

In [744]:
# Re-run the tests above for alpha in [0, 1, 2, 4, 8]
alphas = [0, 1, 2, 4, 8]

for filename in ["16.txt", "25.txt", "50.txt", "64.txt"]:
    A_original = load_as_csr(filename)
    
    for alpha in alphas:
        A = add_to_diagonal(A_original, alpha)
        b = np.random.default_rng(0).random(num_rows(A))
        print(f"For matrix {filename} with alpha = {alpha}:")
        gauss_seidel(A, b)      

#What happens for larger 𝛼? Can you explain this in terms of the eigenvalues of the matrix?
# As the value of the alpha becomes larger, so does the entries for the diagonal matrix. This causes a slower convergence for the eigenvalues. 
# For gauss-seidel, the eigenvalues in the coefficient matrix and iteration matrix both work hand-in-hand. So when eigenvalues of changed matrix increase,
# due to a larger alpha, this can also lead to divergence. A slower convergence and divergence both have chances of occurring in this case.

For matrix 16.txt with alpha = 0:
Converged after 136 iterations
For matrix 16.txt with alpha = 1:
Converged after 80 iterations
For matrix 16.txt with alpha = 2:
Converged after 57 iterations
For matrix 16.txt with alpha = 4:
Converged after 38 iterations
For matrix 16.txt with alpha = 8:
Converged after 23 iterations
For matrix 25.txt with alpha = 0:
Converged after 71 iterations
For matrix 25.txt with alpha = 1:
Converged after 6 iterations
For matrix 25.txt with alpha = 2:
Converged after 5 iterations
For matrix 25.txt with alpha = 4:
Converged after 4 iterations
For matrix 25.txt with alpha = 8:
Converged after 4 iterations
For matrix 50.txt with alpha = 0:
Converged after 115 iterations
For matrix 50.txt with alpha = 1:
Converged after 6 iterations
For matrix 50.txt with alpha = 2:
Converged after 5 iterations
For matrix 50.txt with alpha = 4:
Converged after 5 iterations
For matrix 50.txt with alpha = 8:
Converged after 4 iterations
For matrix 64.txt with alpha = 0:
Converged af